In [1]:
#Import all relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.express as px
import plotly.graph_objects as go
from ydata_profiling import ProfileReport
from plotly.subplots import make_subplots

## This statement allows the visuals to render within your Jupyter Notebook.
%matplotlib inline



# **Laod the data** 

In [2]:
df=pd.read_csv("C:\\Users\\ryyao\\Downloads\\quarter_report SI.CSV")

### **Profinling the data**

In [3]:
df.head(2)

,yearnumber,quarternumber,quarternamear,quarterid,region_ar,city_ar,district_ar,typecategoryar,deed_counts,RealEstatePrice_SUM,Meter_Price_W_Avg_IQR
0,2023,1,الربع الأول,20231,منطقة الرياض,الرياض,الأندلس,أخرى,1,120000.0,NaN
1,2018,3,الربع الثالث,20183,منطقة الرياض,الرياض,الأندلس,أخرى,1,1000000.0,NaN


In [4]:
df.shape

(32730, 11)

In [5]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.shape

(32681, 11)

In [6]:
df.isnull().sum()

yearnumber                   0
quarternumber                0
quarternamear                0
quarterid                    0
region_ar                  350
city_ar                    350
district_ar               9222
typecategoryar               0
deed_counts                  0
RealEstatePrice_SUM          0
Meter_Price_W_Avg_IQR    16361
dtype: int64

## **Slicing the data frame**
#### *here we want to slice all the dataset of Riyadh city based on the type of category of the Estate*

In [7]:
riy=df[df["city_ar"]=='الرياض']
riy['Meter_Price_W_Avg_IQR'].isnull().sum()
#print(riy['district_ar'].value_counts())

4740

In [8]:
riy['typecategoryar'].value_counts()

typecategoryar
قطعة أرض-سكنى      3203
فيلا               2615
قطعة أرض-تجارى     2464
شقة                1566
أخرى                707
عمارة               186
قطعة أرض- زراعي      69
Name: count, dtype: int64

In [9]:
riy[(riy['typecategoryar']=='قطعة أرض- زراعي')]['Meter_Price_W_Avg_IQR'].isnull().sum()

64

In [10]:
riy[(riy['typecategoryar']=='فيلا')]['Meter_Price_W_Avg_IQR'].isnull().sum()

1066

In [11]:
riy[(riy['typecategoryar']=='عمارة')]['Meter_Price_W_Avg_IQR'].isnull().sum()

177

In [12]:
riy[(riy['typecategoryar']=='شقة')]['Meter_Price_W_Avg_IQR'].isnull().sum()

629

In [13]:
riy_comm=riy[(riy['typecategoryar']=='قطعة أرض-تجارى')]#['Meter_Price_W_Avg_IQR'].isnull().sum()
riy_comm1=riy_comm[riy_comm['typecategoryar']=='قطعة أرض-تجارى']
riy_comm1.shape

(2464, 11)

In [14]:
riy_comm1.dropna(subset=['Meter_Price_W_Avg_IQR'], inplace=True)
riy_comm1.shape


(1055, 11)

In [15]:
riy_comm1.head(2)

,yearnumber,quarternumber,quarternamear,quarterid,region_ar,city_ar,district_ar,typecategoryar,deed_counts,RealEstatePrice_SUM,Meter_Price_W_Avg_IQR
602,2022,3,الربع الثالث,20223,منطقة الرياض,الرياض,الصحافة,قطعة أرض-تجارى,7,1.374833e+08,13721.804511
604,2021,1,الربع الأول,20211,منطقة الرياض,الرياض,الصحافة,قطعة أرض-تجارى,10,9.041821e+08,9369.218138


In [16]:
risd_land=riy[(riy['typecategoryar']=='قطعة أرض-سكنى')]
risd_land.shape

(3203, 11)

In [17]:
exclude_categories = ['قطعة أرض- زراعي', 'قطعة أرض-تجارى']
risd_riy = riy[~riy['typecategoryar'].isin(exclude_categories)].copy()
risd_riy['typecategoryar'].value_counts()


In [18]:
agr_riy=riy[(riy['typecategoryar']=='قطعة أرض- زراعي')]

## **Riyadh Regions Slices:**
#### *because we have 145 districts in Riyadh; we will slicing the Riyadh data frame based on Regions*

In [34]:
south=['المصانع','العزيزية','الدار البيضاء','بدر',
        'أحد','طيبة','الغنامية','المناخ','عريض',
        'المصفاة','الحائر'
        ,'المروة','عكاظ','سلطانة',
        'المدينة الصناعية الجديدة','عتيقة'
        ]
west=['الناصرية','عرقة','ظهرة لبن','طويق','ديراب'
      ,'ظهرة البديعة','نمار','السويدي الغربي'
         ,'العريجاء الغربي','السويدي','العريجاء',
        'أم الحمام الغربي','شبرا','لبن','الفاخرية'
        ,'ضاحية نمار',' العريجاء الأوسط'
        ,'البديعة','المعذر','الرفيعة'
        ,'العوالي','الحزم','الزهرة','الدريهمية','المهدية'
        ,'العريجاء الأوسط'
        ]
centar=['غبيرا','العمل','الصناعية',' منفوحة الجديدة',
        'جرير','اليمامة',
        'الجزيرة','منفوحة','الوزارات','أم سليم','الديرة',
        'العود','الضباط','المربع','الخالدية','  الملك عبدالعزيز',
        'الفيصلية','الهدا','معكال '
        ,'الشميسي','منفوحة الجديدة'
        ,'الرفيعة'
        ,'الزهراء','الصالحية','الملز','معكال'
        'الفاروق ','المرقب','الجرادية', 'ثليم', 
         
          ]
north=['الفلاح','الوادي','الندى','الربيع'
       ,'النفل','الغدير','الصحافة','العقيق'
       ,'حطين','الملقا','الياسمين','النرجس'
       ,'العارض','القيروان','بنبان',''
       ,'الخير','مطار الملك خالد الدولي','العقيق',
       'الياسمين','الصحافة','الربيع','الغدير','النفل'
       ,'الوادي','الندى','الفلاح','الإزدهار'
       ,'النخيل','المروج','المصيف','التعاون'
       ,'الازدهار','مغرزات','النزهة','المرسلات'
       ,'المحمدية','الملك فهد','الرحمانية','الورود'
       ,'الرائد','الخزامى','الواحة','صلاح الدين'
       ,'الملك عبدالله','السليمانية','العليا','المعذر الشمالي'
       ,'أم الحمام الشرقي','المعذر',''
       ]
east=['الرمال','المونسية','قرطبة','البيان'
      ,'المرجان','الوسام','السحاب','المشرق'
      ,'الزاهر','التضامن','الشعلة','العلا'
      ,'الزهور','الراية','الفرسان','الدانة'
      ,'الرسالة','الرسالة','الرحاب','المجد'
      ,'النخبة','الشهداء','غرناطة','اليرموك'
      ,'القادسية','الجنادرية','النظيم','الحمراء'
      ,'إشبيلية','القدس','الملك فيصل','الخليج'
      ,'الروضة','الأندلس','النهضة','المعيزيلة'
      ,'الندوة','الربوة','الريان','المنار'
      ,'النسيم الغربي','النسيم الشرقي','الرماية','الروابي'
      ,'السلام','الجزيرة','الفيحاء','السعادة'
      ,'خشم العان','السلي','المشاعل','البرية'
      ]
means=riy.pivot_table(values='Meter_Price_W_Avg_IQR', index='district_ar', columns='yearnumber', aggfunc='mean').reset_index()
risd_mean = risd_land.pivot_table(values='Meter_Price_W_Avg_IQR', index='district_ar', columns='yearnumber', aggfunc='mean').reset_index()
comm_mean = riy_comm1.pivot_table(values='Meter_Price_W_Avg_IQR', index='district_ar', columns='yearnumber', aggfunc='mean').reset_index()
land_mean = risd_land.pivot_table(values='Meter_Price_W_Avg_IQR', index='district_ar', columns='yearnumber', aggfunc='mean').reset_index()

# **Now lets start visualizion**

#### *we will display the visualizes Estates in tow catigories : RISDUALS and COMMERCIAL* 

In [35]:
fig = go.Figure()
lis=centar
for district in lis:
    district_data = land_mean[land_mean['district_ar'] == district]
    if not district_data.empty:
        fig.add_trace(go.Scatter(x=district_data.columns[1:], y=district_data.values[0][1:], mode='lines+markers', name=district))

fig.update_layout(
    title='RISDUALS Estates Changes for Different Districts ',
    xaxis_title='Year',
    yaxis_title='Meter Price',
    legend_title='District',
    width=800,
    height=600
)
fig.show()


In [21]:
fig = go.Figure()
lis=north
for district in lis:
    district_data = comm_mean[comm_mean['district_ar'] == district]
    if not district_data.empty:
        fig.add_trace(go.Scatter(x=district_data.columns[1:], y=district_data.values[0][1:], mode='lines+markers', name=district))

# Update layout
fig.update_layout(
    title='COMMERCIAL Estates Changes for Different Districts ',
    xaxis_title='Year',
    yaxis_title='Meter Price',
    legend_title='District',
    width=800,
    height=600
)
fig.show()










# **Extra Questions and Futur work :**
## Q : WHO LEAD THE ESTATE MARKET ?**
#### IS IT RISDUALS OR COMMERCIAL ??
#### *we will study the relation between RISDUALS Estates & COMMERCIAL Estates , in terms of the time to discover who lead the Market...*


In [22]:
risd18=risd_land[risd_land['yearnumber']==2018]
risd19=risd_land[risd_land['yearnumber']==2019]
risd20=risd_land[risd_land['yearnumber']==2020]
risd21=risd_land[risd_land['yearnumber']==2021]
risd22=risd_land[risd_land['yearnumber']==2022]
risd23=risd_land[risd_land['yearnumber']==2023]



In [23]:
risd18=risd18[['quarternumber','Meter_Price_W_Avg_IQR']]
risd19=risd19[['quarternumber','Meter_Price_W_Avg_IQR']]
risd20=risd20[['quarternumber','Meter_Price_W_Avg_IQR']]
risd21=risd21[['quarternumber','Meter_Price_W_Avg_IQR']]
risd22=risd22[['quarternumber','Meter_Price_W_Avg_IQR']]
risd23=risd23[['quarternumber','Meter_Price_W_Avg_IQR']]

In [26]:
profile = ProfileReport(risd_land, title="Profiling Report", explorative=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
fig = go.Figure()
lis=north
for district in lis:
    district_data = comm_mean[comm_mean['district_ar'] == district]
    if not district_data.empty:
        fig.add_trace(go.Scatter(x=district_data.columns[1:], y=district_data.values[0][1:], mode='lines+markers', name=district))

# Update layout
fig.update_layout(
    title='COMMERCIAL Estates Changes for Different Districts ',
    xaxis_title='Year',
    yaxis_title='Meter Price',
    legend_title='District',
    width=800,
    height=600
)
fig.show()